In [ ]:
# BIKES and NEXTBIKE STATIONS

import os, os.path, datetime, json

# initialize sets for bikes (both providers)
callabike_set = set()
nextbike_set = set()
# initialize an set (only IDs) and a list for nextbike stations
nextbike_stations_id_set = set()
nextbike_stations_list = list()

'''
for day in range(4,5):
    for hour in range(0,1):
        for minute in range(0,1):
'''

# for all minutes within the given days
for day in range(5,20):
    for hour in range(0,24):
        for minute in range(0,60):
            # format date and time strings (YYYY-MM-DD and HH-MM), append to directory path
            date = '2021-06-' + f'{day:02d}'
            time = f'{hour:02d}' + '-' + f'{minute:02d}'
            DIR = 'json/' + date + '/' + time;
            # if given directory exists
            if(os.path.exists(DIR)):
                
                # NEXTBIKE
                # if given nextbike JSON exists in directory
                if(os.path.exists(DIR+'/nextbike.json')):
                    try:
                        # open JSON and load "places" subarray into array
                        with open(DIR+'/nextbike.json') as bike_json:
                            jsonObject = json.load(bike_json)
                            bike_json.close()
                        places = jsonObject['countries'][0]['cities'][0]['places']
                        # for all places (could be either bikes or stations)
                        for place in places:
                            # BIKES
                            # go through bikes in bike_list array of place
                            for bike in place['bike_list']:
                                # retrieve number from bike and add to nextbike set
                                bike_number = bike['number'];
                                nextbike_set.add(bike_number);
                            # STATIONS
                            # if place is a station ('spot' = true)
                            if (place['spot']):
                                # retrieve place uid and check if already exists in ID set
                                place_id =  place['uid']
                                if(place_id not in nextbike_stations_id_set):
                                    # if ID not yet in ID set: add ID to set
                                    nextbike_stations_id_set.add(place_id)

                                    # retrieve coordinates, name and capacity of station
                                    place_latitude = str(place['lat'])
                                    place_longitude = str(place['lng'])
                                    # format string for MySQL: ST_GeomFromText with SRID 2348 requires Point(52.53153 13.38651) -> coordinates are in reverse order!
                                    place_coordinates = 'Point(' + place_latitude + ' ' + place_longitude + ')'
                                    place_name = place['name']
                                    place_provider =  'nextbike'
                                    place_capacity = place['bike_racks']

                                    # add station data to station list
                                    nextbike_stations_list.append(list([place_id,place_coordinates,place_name,place_provider,place_capacity]))
                    
                    # catch JSON errors and print them with file path
                    except ValueError as e:
                        print(DIR + ': ' + 'JSONDecodeError')
                        # Exception sample: 2020-06-17/00-58, 2020-06-19/13-05, 2021-06-20/00-56, 2021-06-20/03-50:
                    except KeyError as e:
                        print(DIR + ': ' + 'KeyError')
                    except IndexError as e:
                        print(DIR + ': ' + 'IndexError')
                    except Error as e:
                        print(e)

                # CALL-A-BIKE
                # go through numbers 0-18
                for callabike_json_number in range(0,19):
                    # if call-a-bike JSON with given number exists
                    json_path = DIR+'/callabike-'+str(callabike_json_number)+'.json'
                    if(os.path.exists(json_path)):
                        try:
                            # open JSON and write subarray 'items' to array bikes
                            with open(json_path) as bike_json:
                                jsonObject = json.load(bike_json)
                                bike_json.close()
                            bikes = jsonObject['items']
                            # if variable 'href_sample' does not exist
                            if(not 'href_sample' in vars()):
                                # take rentalObject href of first bike
                                href_sample = str(bikes[0]['rentalObject'])
                                # find out beginning and end position of ID within that href
                                left_cut = href_sample.rfind("rentalobjects/")+14
                                right_cut = href_sample.rfind("'}'")-1
                            for bike in bikes:
                                bike_href = str(bike['rentalObject'])
                                # cut bike ID out of href
                                bike_id = bike_href[:right_cut][left_cut:]
                                # add ID to call-a-bike set
                                callabike_set.add(bike_id)
                        # catch JSON errors and print them with file path
                        except ValueError as e:
                            print(json_path + ': ' + 'JSONDecodeError')
                        except KeyError as e:
                            print(json_path + ': ' + 'KeyError')
                        except Error as e:
                            print(e)

print(nextbike_set)
print(callabike_set)

In [10]:
print(len(callabike_set))
print(len(nextbike_set))
print(len(nextbike_stations_list))
#print(json.dumps(nextbike_stations_list,indent=5))

1671
3844
321


In [7]:
# BIKES IN DATABASE

from mysql.connector import connect, Error
from getpass import getpass

# put call-a-bike and nextbike lists into lists with IDs as individual arrays
callabike_list = [list([bike_id]) for bike_id in list(callabike_set)]
nextbike_list = [list([bike_id]) for bike_id in list(nextbike_set)]

# create MySQL connection
connection = connect(
        host="localhost",
        user="root",
        password=getpass("Enter password: "),
        database="bikes",
    )

try:
    # generate cursor and insert both lists into bike table, "skipping" duplicates by updating them with their own key
    cursor = connection.cursor()
    cursor.executemany("INSERT INTO bike (bike_id, provider) VALUES (%s, 'callabike') ON DUPLICATE KEY UPDATE bike_id=bike_id", callabike_list)
    cursor.executemany("INSERT INTO bike (bike_id, provider) VALUES (%s, 'nextbike') ON DUPLICATE KEY UPDATE bike_id=bike_id", nextbike_list)
    # result = print(cursor.fetchone())
    # print(result[0])
    connection.commit()
except Error as e:
        print(e)


In [ ]:
# STATIONS CALL-A-BIKE

import os, os.path, datetime, json

callabike_stations_list = list()
nextbike_stations_list = list()

DIR = 'json/Stations';
# go through files 1 to 4
for j in range(1,5):
    try:
        with open(DIR+'/CallABike'+str(j)+'.json') as station_json:
            jsonObject = json.load(station_json)
            station_json.close()
        stations = jsonObject['items']
        # for every station 
        for station in stations:
            # retrieve station data and add to call-a-bike station list
            station_id =  station['uid']
            station_latitude = str(station['geometry']['centroid']['coordinates'][1])
            station_longitude = str(station['geometry']['centroid']['coordinates'][0])
            # Format for MySQL ST_GeomFromText and SRID 4326: Point(52.53153 13.38651) -> in reverse order!
            station_coordinates = 'Point(' + station_latitude + ' ' + station_longitude + ')'
            station_name = station['name']
            station_provider =  'callabike'
            callabike_stations_list.append(list([station_id,station_coordinates,station_name,station_provider]))
    except Error as e:
        print('Error occured!')
print(callabike_stations_list)

In [9]:
# STATIONS IN DATABASE

from mysql.connector import connect, Error
from getpass import getpass

# create MySQL connection
connection = connect(
        host="localhost",
        user="root",
        password=getpass("Enter password: "),
        database="bikes",
    )

try:
    cursor = connection.cursor()
    # insert stations -> coordinates with SRID 4326, otherwise lat and lon are reversed!
    cursor.executemany("INSERT INTO station (station_id, coordinates, name, provider) VALUES (%s,  ST_GeomFromText(%s, 4326), %s, %s) ON DUPLICATE KEY UPDATE station_id=station_id", callabike_stations_list)
    cursor.executemany("INSERT INTO station (station_id, coordinates, name, provider, capacity) VALUES (%s,  ST_GeomFromText(%s, 4326), %s, %s, %s) ON DUPLICATE KEY UPDATE station_id=station_id", nextbike_stations_list)
    # result = print(cursor.fetchone())
    # print(result[0])
    connection.commit()
except Error as e:
        print(e)


None


TypeError: 'NoneType' object is not subscriptable

In [52]:
# interrupt faulty SQL query
connection.commit()